In [1]:
from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.operators import OnePointCrossOver, GaussianMutation
from torch import Tensor
import random
import torch
import math

In [2]:
def distance(x, y):
    return math.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2)
    
def fun(x):
    #Huge penalty for non unique indexes
    uniq = x.unique()
    if(uniq.size(0) != x.size(0)):
        return (x.size(0)-uniq.size(0))*penalty
    dist = distance(cityList[x[x.size(0)-1]], cityList[x[0]])
    for i in range(x.size(0) - 1):
        dist += distance(cityList[x[i]], cityList[x[i + 1]])
    return dist

def mutation(values: torch.Tensor) -> torch.Tensor:
    #Makes indexes all unique
    if(values.unique().size(0) != values.size(0)):
        t = list(range(city_count))
        random.shuffle(t)
        values = torch.Tensor(t)

    #Swaps values in population randomly
    swap_count = math.floor(city_count*0.05)
    for i in range(swap_count):
        r1 = random.randint(0, city_count-1)
        r2 = random.randint(0, city_count-2)
        if(r2 >= r1):
            r2 += 1
        values[r1], values[r2] = values[r2], values[r1]
    return values
def create_city_list(city_count: int) -> list:
    val_range = city_count*1000
    return [
        (random.randrange(val_range), random.randrange(val_range))
        for _ in range(city_count)
    ]

In [3]:
max_dist = 0
popsize = 100
city_count = 5
cityList = create_city_list(city_count)

for i in cityList:
    for j in cityList:
        dist = distance(i, j)
        if(dist > max_dist):
            max_dist = dist
penalty = max_dist * len(cityList)

In [4]:
problem = Problem("min", fun, bounds=(0, city_count-1), solution_length=city_count, dtype=torch.int64)

searcher = GeneticAlgorithm(problem, popsize=popsize, operators=[
    OnePointCrossOver(problem, tournament_size=math.floor(city_count*0.1)),
    mutation
])

[2024-05-28 20:49:46] INFO     <11429> evotorch.core: Instance of `Problem` (id:140516275224656) -- The `dtype` for the problem's decision variables is set as torch.int64
[2024-05-28 20:49:46] INFO     <11429> evotorch.core: Instance of `Problem` (id:140516275224656) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2024-05-28 20:49:46] INFO     <11429> evotorch.core: Instance of `Problem` (id:140516275224656) -- The `device` of the problem is set as cpu
[2024-05-28 20:49:46] INFO     <11429> evotorch.core: Instance of `Problem` (id:140516275224656) -- The number of actors that will be allocated for parallelized evaluation is 0


In [5]:
searcher.run(1000)

IndexError: argmax(): Expected reduction dim 1 to have non-zero size.

In [ ]:
searcher.status